<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.981 · TFM · Aula 1</p>
<p style="margin: 0; text-align:right;">2024-2 · Màster universitari en Ciència de dades (Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudis d'informàtica, multimèdia i telecomunicació</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>
<div style="text-align: center; margin-top: 40px;">
    <h1>Aplicación para ayuda a la conducción con deep learning</h1>
</div>

### ARCHIVO 10: Entrenamiento frenos

Entrena un modelo clasificador binario.

Código que ejecuta el entrenamiento a partir de imágenes en carpetas definidas. Modelo a entrenar: EfficientNet B0.

Código ejecutado en plataforma Visual Studio Code.

In [2]:
# Llibreries

import os
import cv2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch.nn.functional as F

In [ ]:
# Dades

Dades = "Llums_Cotxe" # Carpeta amb subcarpetes per classe

# Bloc per a la normalització d'imatges
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [ ]:
# Classe per a crear un dataset

class BrakeDataset(Dataset):
    def __init__(self, paths, labels):
        self.paths = paths
        self.labels = labels

    def __len__(self): return len(self.paths)

    def __getitem__(self, i):
        img = cv2.imread(self.paths[i])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = transform(img)
        return img, torch.tensor(self.labels[i], dtype=torch.float32)

In [ ]:
# Llegeix carpetes

# Funció per a recopilar les imatges i torna datasets separats
def get_data():
    paths, labels = [], []
    for label, folder in enumerate(['NoFreno', 'Freno']):
        full = os.path.join(Dades, folder)
        for f in os.listdir(full):
            if f.lower().endswith(('jpg', 'png', 'jpeg')):
                paths.append(os.path.join(full, f))
                labels.append(label)
    return train_test_split(paths, labels, test_size=0.2, random_state=42)

# Dades i loaders
train_paths, test_paths, train_labels, test_labels = get_data()
train_loader = DataLoader(BrakeDataset(train_paths, train_labels), batch_size=16, shuffle=True)

In [ ]:
# Càrrega model

model = models.efficientnet_b0(pretrained=True)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 1)
model = model.cuda()


# Paràmetres
loss_fn = nn.BCEWithLogitsLoss()
opt = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
# Entrenament

model.train()
for epoch in range(5):
    total_loss = 0
    for x, y in train_loader:
        x, y = x.cuda(), y.unsqueeze(1).cuda()
        out = model(x)
        loss = loss_fn(out, y)
        opt.zero_grad()
        loss.backward()
        opt.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}: {total_loss/len(train_loader):.4f}")

torch.save(model, "DetectBrake3.pt")

# Avaluació
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for x, y in DataLoader(BrakeDataset(test_paths, test_labels), batch_size=16):
        x = x.cuda()
        y_true.extend(y)
        out = model(x)
        probs = torch.sigmoid(out).cpu()
        y_pred.extend((probs > 0.5).int().squeeze().tolist())

# Mètriques
print("Mètriques de validació:")
print("Accuracy :", accuracy_score(y_true, y_pred))
print("Precision:", precision_score(y_true, y_pred))
print("Recall   :", recall_score(y_true, y_pred))
print("F1 Score :", f1_score(y_true, y_pred))

print("Fi d'entrenament")

c:\Users\malve\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\malve\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1: 0.6083
Epoch 2: 0.4083
Epoch 3: 0.2647
Epoch 4: 0.1498
Epoch 5: 0.1041
Mètriques de validació:
Accuracy : 0.9545454545454546
Precision: 0.92
Recall   : 0.92
F1 Score : 0.92
Fi d'entrenament
